In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import full_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# To create genre-specific npz files (required for randomizing splits)
#fileio.process_data_into_np_files()

# To create 27 random splits, each of size 32. (Except the last one, data/working/splits/split_26.npz is only size 5)
#fileio.create_random_splits() # Randomized with random.seed(1), so our splits will be identical

# To load a range of splits
#X, Y = fileio.load_splits(range(5)) # loads splits 0-4
#X.shape = (fileio.NUM_SAMPLES_GTZAN, 5 * 32)
#Y.shape = (24, 5 * 32)

# To generate chroma, aligned with the splits
#fileio.generate_chroma_splits_from_splits()

# To load a range of chroma splits
#X, Y = fileio.load_chroma_splits(range(5)) # loads chroma splits 0-4
#X.shape = (300, 12, 5 * 32)
#Y.shape = (24, 5 * 32)

In [3]:
def get_scores(targets, guesses):
    scores = []
    
    for idx in range(len(targets)):
        scores.append(get_score_single(targets[idx], guesses[idx, :]))
        
    return scores
    

In [4]:
def get_score_single(target, guess):
    scores = keys.get_vector_from_key(target) * 2
    return scores[keys.generate_one_hot_guess(guess.data)]

In [8]:
def load_torch_data_from_matrices(X, Y):
    X = torch.transpose(torch.from_numpy(X), 0, 2)
    Y = torch.from_numpy(keys.generate_one_hot_matrix(Y))

    X = X.float()
    Y = Y.long()
    
    return X, Y

In [ ]:
import importlib

importlib.reload(full_model)

net = full_model.Net()
print(net)

X, Y = fileio.load_chroma_splits(range(20))
X, Y = load_torch_data_from_matrices(X, Y)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

for epoch in range(5000):
    optimizer.zero_grad()
    
    Y_hat = net(X)
    loss = criterion(Y_hat, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('epoch ' + str(epoch) + ' loss: ' + str(loss.item()))
        print('Accuracy = ' + str(np.mean(get_scores(Y, Y_hat))))
        
print('Done')

Net(
  (conv1): Conv1d(12, 32, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=32, out_features=24, bias=True)
)
epoch 0 loss: 3.1782712936401367
Accuracy = 0.0459375
epoch 100 loss: 3.1353890895843506
Accuracy = 0.2809375
epoch 200 loss: 3.0236074924468994
Accuracy = 0.36968750000000006
epoch 300 loss: 2.9198648929595947
Accuracy = 0.538125
epoch 400 loss: 2.830703020095825
Accuracy = 0.5985937499999999
epoch 500 loss: 2.785581588745117
Accuracy = 0.6078125
epoch 600 loss: 2.7631564140319824
Accuracy = 0.6075
epoch 700 loss: 2.7490956783294678
Accuracy = 0.6059375
epoch 800 loss: 2.7388038635253906
Accuracy = 0.61328125
epoch 900 loss: 2.7305259704589844
Accuracy = 0.62265625
epoch 1000 loss: 2.723646640777588
Accuracy = 0.63015625
epoch 1100 loss: 2.7177462577819824
Accuracy = 0.63375
epoch 1200 loss: 2.712559461593628
Accuracy = 0.63671875
epoch 1300 loss: 2.7079310417175293
Accuracy = 0.63890625
epoch 1400 loss: 2.7038049697875977
Accuracy = 0.64125
epoch 1500 loss: 2.70

In [ ]:
# Train accuracy
np.mean(get_scores(Y, net(X)))

In [ ]:
# Test accuracy
X, Y = fileio.load_chroma_splits(range(20,27))
X, Y = load_torch_data_from_matrices(X, Y)

np.mean(get_scores(Y, net(X)))